In [ ]:
%pylab inline
import os
import pickle
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import keras
from keras.layers import convolutional, Dense, Activation,pooling
from keras.layers import Input, merge, Convolution2D, MaxPooling2D, UpSampling2D, Flatten, Dropout
from keras.optimizers import Adam, SGD
from keras.models import Sequential, Model
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras.regularizers import l2
from keras import backend as K
from keras.utils import np_utils
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
import cPickle as pickle
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import h5py


config = tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = 1.0
set_session(tf.Session(config=config))


In [ ]:
###################################### LOADING DATA ##############################

#USER INTERFACE 

### Create a dictionary that contains all the dictiories within each dataset (.mat file) (workspace) we want to load ###
#SELECT data directory
dataDir = input("Enter data directory path: ")  #dataDir = "/scratch/barbieri/DATA_CNN_networks/Data/"
# "/scratch/barbieri/DATA_CNN_networks/data_pickle/"


#Load data
Load_Data = {}  
for i in range(0, len(os.listdir( dataDir ))):  
    Load_Data[i] = {}  #Dictionary for a single workspace


In [ ]:
def CNN_parameters():
    # Please edit parameters
    
    ## =========================================================================
    # Parameters for the CNN
    
    # Define dictionary for the parameters
    PARAMETERS = {}
    
    # Number of classes
    PARAMETERS['n_classes'] = 6  #3 or 6
    
    # Choose Dropout value
    PARAMETERS['dropout'] = 0.4
    
    # Define number of epochs
    PARAMETERS['n_epoch'] = 4 #800
         
    # Define batch size
    PARAMETERS['batch_size'] = 1 #50
            
    # Define decay value to be tested: write in array format
    PARAMETERS['decay_value'] = [0.0001]
    
    # Define learning rate
    PARAMETERS['learning_rate'] = 0.001
    
    # Define the number of iteration per CNN
    PARAMETERS['n_iteration'] = 2 #5
            
    ## =========================================================================
    # Parameters for the data
            
    # Choose the number of curves to use as training data
    PARAMETERS['n_training_data'] = 1800

    # Choose number of point to downsample an intensity curve
    PARAMETERS['points_curve'] = 128
    
    return PARAMETERS

In [ ]:
def data_dict(Load_Data):
    
    #### Define another dictionary for storing intensity curves and classes ### 
    #From each dataset, the intensity curve and classes are extracted and stored in another dictionary
    DATA = {}
    N_of_file =len(os.listdir( dataDir ))
    for i in range(0, N_of_file):
            DATA[i] = {}

    
    #Find the minimun length among all the intensity curves
    Int_curve_lengths = np.zeros(N_of_file)
    for i in range(0, N_of_file):
        Int_curve_lengths [i] = (len(Load_Data[i]['RAWDATA'][0,0]['files'][0,0][0]['intensity'][0][0]))
    Int_curve_len = (min(Int_curve_lengths.astype(int)))


    #### Fill the dictionary ###
    Ncell_classified = 200 #Number of classified cells per dataset
    for i in range(0, N_of_file):
        #Decision array for each dataset
        DATA[i]['decision'] = Load_Data[i]['decisionlist']   
        #Size of the matrix to store intensity values
        NumofData = len(Load_Data[i]['RAWDATA'][0,0]['files'][0,0][0]['intensity'])
        #Empty matrix to store all the intensity values of all the cells
        DATA[i]['intensity'] = np.zeros((Ncell_classified, Int_curve_len))    
        for k in range(0, Ncell_classified): #Only the first 200 were classified
            DATA[i]['intensity'][k,:] = Load_Data[i]['RAWDATA'][0,0]['files'][0,0][0]['intensity'][k][0,0:900] #copy the first 900 values


    ### Classes array: contain values from all the data ###
    Classes=[]
    for i in range( 0, N_of_file ):
        Classes = np.append(Classes , DATA[i]['decision']).astype(int)


    ### Create matrix to store intensity values coming from all the data ###    
    NTot_Data = N_of_file*Ncell_classified
    Intensity = np.zeros((0, DATA[0]['intensity'].shape[1]))
    for i in range( 0, N_of_file ):
        Intensity = np.append(Intensity, DATA[i]['intensity'][:,:], axis=0)    

    ### ELIMINATE intensity profiles with NaN values and corresponding classes ###
    nan_indices = np.where(np.isnan(Intensity))
    Intensity = np.delete(Intensity,nan_indices[0], axis = 0)
    Classes = np.delete(Classes,nan_indices[0], axis = 0)
    print('Intensity curve shape =', Intensity.shape)
    print('Classes shape =', Classes.shape)
    #Redefine the total number of data
    total = len(Intensity)

    return Intensity, Classes, Int_curve_len, total

In [ ]:
# DUPLICATE Intensity profiles adding noise to the second group
def duplicate_data(total, Intensity, Classes, Int_curve_len):

    Intensity_duplicated_noise = np.zeros((2*total, Int_curve_len ))
    Intensity_duplicated_noise[0:total,:] = Intensity
    Intensity_duplicated_noise[total:2*total,:] = Intensity + np.random.randn(Int_curve_len)*0.02

    plt.figure(1)
    plt.subplot(211)
    plt.title('Calcium signalling without and with noise added')
    plot(Intensity_duplicated_noise[1])
    plt.subplot(212)
    plot(Intensity_duplicated_noise[991])
    plt.xlabel('Frame')
    #plt.savefig('13_06_2017_Curve_VS_noiseCurve.png')
    plt.show()

    #Redefine the total number of data
    total = len(Intensity_duplicated_noise)

    #DUPLICATE CLASSES
    Classes_duplicated = np.append(Classes, Classes)
    print('total classes = ', Classes_duplicated.shape)
    
    return Intensity_duplicated_noise, Classes_duplicated, total

In [ ]:
#Perform the DOWNSAMPLING of the data: 256 or 128 points
def downsample(Intensity, PARAMETERS):
    
    if PARAMETERS['points_curve'] == 256:   
        data1 = Intensity[:,0:768]
        step = 3
        data_downsample = data1[:,::step]
        #Redefine len of the Intensity curves:
        Int_profile_len = len(data_downsample[0])        
    elif PARAMETERS['points_curve'] == 128:
        data1 = Intensity[:,0:896]
        step = 7
        data_downsample = data1[:,::step]
        #Redefine len of the Intensity curves:
        Int_profile_len = len(data_downsample[0])    
  
    
    return data_downsample, Int_profile_len

In [ ]:
def run_CNN(Data, Classes, total, Name_file, PARAMETERS):
    
    #Matrix to store test_cl and test_dt  
    n_testing_data = total - PARAMETERS['n_training_data']

    test_dt = np.zeros(((PARAMETERS['n_iteration'] , n_testing_data, PARAMETERS['points_curve'] )))  #180 data, 256 points for each data, 5 iterations
    test_cl = np.zeros(( PARAMETERS['n_iteration'], n_testing_data ))
    
    #Dictionary to store the restults after each iteration
    RESULTS = {} 
    for iteration in range(PARAMETERS['n_iteration']):
        RESULTS[iteration] = {} #dictionary to store the results from the CNN
    
    #Dictionary to store the predictions (out)
    out = {} 
    for iteration in range(PARAMETERS['n_iteration']):
        out[iteration] = {} #dictionary to store the results from the CNN

    #Run the CNN for a certain number of iterations
    for iteration in range(PARAMETERS['n_iteration']):
        print('Iteration: ', iteration)
        
        ################### Distribute all values randomly ##################   
        randseq = np.random.choice(np.arange(0,total),total,replace=False)
        
        #Data randomly organized
        data_random = np.copy(Data)  
        data_random = data_random #[randseq,:]
        
        #Respective Classes 
        Classes_random = np.copy(Classes)
        Classes_random = Classes_random #[randseq]   
        
        #EXAMPLE
        # Data - inputs, 10000 samples of 128-dimensional vectors
        # Classes - labels, 10000 samples of scalars from the set {0, 1, 2}

        # process the data to fit in a keras CNN properly
        # input data needs to be (N, C, X, Y) - shaped where
        # N - number of samples
        # C - number of channels per sample
        # (X, Y) - sample size

        #train_dt_reshape = train_dt_reshape.reshape((10000, 128,1, 1))
        # output labels should be one-hot vectors - ie,
        # 0 -> [1, 0, 0]
        # 1 -> [0, 1, 0]
        # 2 -> [0, 0, 1]
        # this operation changes the shape of y from (10000,1) to (10000, 3)       
        
        
        ######################### load file - TRAINING data ####################
        #Data

        train_dt = data_random[0:PARAMETERS['n_training_data'],:]
        X = np.copy(train_dt)   
        #Classes
        train_cl = Classes_random[0:PARAMETERS['n_training_data']]        
        y = np.copy(train_cl) 


        # process the data to fit in a keras CNN properly
        # input data needs to be (N, C, X, Y) - shaped where
        # N - number of samples
        # C - number of channels per sample
        # (X, Y) - sample size
        X = X.reshape((PARAMETERS['n_training_data'], Int_profile_len,1, 1))
        y = np_utils.to_categorical(y)

        ######################### load file - TESTING data ####################

        #Data
        test_dt[iteration, :, :]= data_random[PARAMETERS['n_training_data']:total,:]        
        test_dt_reshape = np.copy(test_dt[iteration,:,:])


        #Classes
        test_cl[iteration,:] = Classes_random[PARAMETERS['n_training_data']:total] 
        test_cl_reshape = np.copy(test_cl[iteration,:])

        
        # process the data to fit in a keras CNN properly
        # input data needs to be (N, C, X, Y) - shaped where
        # N - number of samples
        # C - number of channels per sample
        # (X, Y) - sample size    
        test_dt_reshape = test_dt_reshape.reshape((n_testing_data, Int_profile_len,1, 1)) 
        test_cl_reshape = np_utils.to_categorical(test_cl_reshape)

        
        #RUN THE CNN network
        for i in range(0,len(PARAMETERS['decay_value'])):
            print('decay value = ', PARAMETERS['decay_value'][i])
            RESULTS[iteration][i], out[iteration][i] = CNN(X, y, test_dt_reshape, test_cl_reshape, n_testing_data, Int_profile_len, Name_file, PARAMETERS, i, iteration)


    return RESULTS, out, test_dt, test_cl

In [ ]:
# define a CNN
def CNN(X, y, test_dt_reshape, test_cl_reshape, n_testing_data, Int_profile_len, Name_file, PARAMETERS, i, iteration):

    
    cnn = Sequential()
    cnn.add(Convolution2D(64, 3, 1,border_mode="same",activation="relu",input_shape=(Int_profile_len,1, 1)))
    cnn.add(Convolution2D(64, 3, 1, border_mode="same", activation="relu"))
    cnn.add(MaxPooling2D(pool_size=(2,1)))

    cnn.add(Convolution2D(128, 3, 1, border_mode="same", activation="relu"))
    cnn.add(Convolution2D(128, 3, 1, border_mode="same", activation="relu"))
    cnn.add(Convolution2D(128, 3, 1, border_mode="same", activation="relu"))
    cnn.add(MaxPooling2D(pool_size=(2,1)))

    cnn.add(Convolution2D(256, 3, 1, border_mode="same", activation="relu"))
    cnn.add(Convolution2D(256, 3, 1, border_mode="same", activation="relu"))
    cnn.add(Convolution2D(256, 3, 1, border_mode="same", activation="relu"))
    cnn.add(MaxPooling2D(pool_size=(2,1)))

    cnn.add(Flatten())
    cnn.add(Dense(1024, activation="relu"))
    cnn.add(Dropout(PARAMETERS['dropout'])) #0.5
    cnn.add(Dense(PARAMETERS['n_classes'], activation="softmax"))

    
    # define optimizer and objective, COMPILE cnn
    compile_step = cnn.compile(loss="categorical_crossentropy", optimizer=Adam(lr=PARAMETERS['learning_rate'], beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=PARAMETERS['decay_value'][i]),  metrics=['accuracy'])
    #default parameters coming from the paper: https://arxiv.org/abs/1412.6980v8

    #Store in dictionaries
    results = {}
    out = {}  
    
    #### TRAIN ####
    #_DropOut05_data_weights_1980Data_best_DecayValue_
    print('decay val CNN', PARAMETERS['decay_value'][i])
    filepath ="/scratch/dwaithe/" + str(Name_file)+ "DecValue_" + str(PARAMETERS['decay_value'][i]) + "NIteration_" + str(iteration) + ".hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
     
    
    # FIT the model
    #results = cnn.fit(X, y, batch_size=PARAMETERS['batch_size'], nb_epoch=PARAMETERS['n_epoch'], verbose=0, callbacks=callbacks_list, validation_data=(test_dt_reshape , test_cl_reshape)) 
    results = cnn.fit(X, y, validation_split=0.33, nb_epoch=PARAMETERS['n_epoch'], batch_size=PARAMETERS['batch_size'], callbacks=callbacks_list, verbose=0)
 
    
    
    #EVALUATION code which makes prediction
    out = cnn.predict(test_dt_reshape.reshape((n_testing_data, Int_profile_len,1, 1)))    
    cnn.save('/scratch/dwaithe/'+str(Name_file)+'.h5')
   

    # save THE MODEL STRUCTURE as YAML file
    model_yaml = cnn.to_yaml()
    with open("model.yaml", "w") as yaml_file:
        yaml_file.write(model_yaml)    

    
    return results, out

In [ ]:
#Save LOSS and ACCURACY curve in txt files
def Save(RESULTS, PARAMETERS, Name_file):
    
    #SAVE LOSS    
    for iteration in range(PARAMETERS['n_iteration']):
        for i in range(len(PARAMETERS['decay_value'])):
            np.savetxt("/scratch/dwaithe/"+ str(Name_file)+ "LOSS_Iter_"+ str(iteration)+ "_DecVal_" +str(PARAMETERS['decay_value'][i])+ ".txt", RESULTS[iteration][i].history['loss'], newline='\r\n')

    #SAVE ACCURACY
    for iteration in range(PARAMETERS['n_iteration']):
        for i in range(len(PARAMETERS['decay_value'])):
            np.savetxt("/scratch/dwaithe/" + str(Name_file)+ "ACC_Iter_" + str(iteration)+ "_DecVal_" +str(PARAMETERS['decay_value'][i]) + ".txt", RESULTS[iteration][i].history['acc'], newline='\r\n')

    #SAVE VAL_LOSS
    for iteration in range(PARAMETERS['n_iteration']):
        for i in range(len(PARAMETERS['decay_value'])):
            np.savetxt("/scratch/dwaithe/" + str(Name_file)+ "VAL_LOSS_Iter_"+ str(iteration)+ "_DecVal_"+str(PARAMETERS['decay_value'][i]) + ".txt", RESULTS[iteration][i].history['val_loss'], newline='\r\n')

    #SAVE VAL_ACC
    for iteration in range(PARAMETERS['n_iteration']):
        for i in range(len(PARAMETERS['decay_value'])):
            np.savetxt("/scratch/dwaithe/" + str(Name_file)+ "VAL_ACC_Iter_" + str(iteration)+ "_DecVal_"+str(PARAMETERS['decay_value'][i]) + ".txt", RESULTS[iteration][i].history['val_acc'], newline='\r\n')

    return

In [ ]:
##############################  MAIN  ###################################
import time
start = time.time()

PARAMETERS = CNN_parameters()

#Choose names for files
file_format = input("Enter file format: pickle=0 or mat=1. Format:")
Name_file = input('Insert name of the file for the weights between " " : ')


### Reading Pickle data or .mat data type

################################## PICKLE DATA ########################################
### lOAD THE DATA STORING THEM IN A DICTIONARY
if file_format == 0:
    for file in os.listdir( dataDir ):
        to_export = pickle.load( open( dataDir+file , "rb" ))

    Data = to_export['data'] 
    Classes = to_export['classes']
    total = len(Data)
    Int_profile_len = len(Data[0])   
      
    #Run the CNN
    RESULTS, test_dt, test_cl = run_CNN(Data, Classes, total, Name_file, PARAMETERS)

    
    #Save Loss, Validation Loss, Accuracy, Validation Accuracy
    Save(RESULTS, n_iteration, Name_file)    

############################### .MAT DATA ########################################
elif file_format == 1:    # BE CAREFUL: in spyder3 is '1'
    
    
    ### lOAD THE DATA STORING THEM IN A DICTIONARY ###
    N_of_file =len(os.listdir( dataDir ))
    Load_Data = {}  
    for i in range(0, N_of_file):  
        Load_Data[i] = {}  #Dictionary for a single workspace
    
    i = 0
    for file in os.listdir( dataDir ):
        Load_Data[i] =  scipy.io.loadmat( dataDir+file )
        i = i+1    
    
    #### Define another dictionary for storing intensity curves and classes ### 
    Intensity, Classes, Int_curve_len, total = data_dict(Load_Data)
    
    ### Duplicate data adding noise ### 
    answer = input('Do you want to duplicate the curves adding noise? Yes = 1; No = 0. Answer: ')
    if answer == 1:
        Intensity, Classes, total = duplicate_data(total, Intensity, Classes, Int_curve_len)
        n_training_data = PARAMETERS['n_training_data']        
    elif answer == 0:
        n_training_data = PARAMETERS['n_training_data'] #input('Number of data to use for training the CNN network: ') #800
       
    ### Downsampling
    data_downsample, Int_profile_len = downsample(Intensity, PARAMETERS)
    #fig, ax = plt.subplots()
    #ax.plot(data_downsample[16])
    
    #Check if there are typo errors for the classes (we only have 6 classes going from 0 to 5)
    for i in range(len(Classes)):
        if Classes[i] > 5:
            print('more classes', i)
            
    
    #Run the CNN
    RESULTS, out, test_dt, test_cl = run_CNN(data_downsample, Classes, total, Name_file, PARAMETERS)
                            
    #Save Loss, Validation Loss, Accuracy, Validation Accuracy 
    Save(RESULTS, PARAMETERS, Name_file)
    
    end = time.time()
    print('Time = ', end - start)

In [ ]:
LOSS ={}
VAL_LOSS = {}
for k in range(len(PARAMETERS['decay_value'])):
    LOSS[k] = np.zeros((PARAMETERS['n_iteration'], PARAMETERS['n_epoch'])) #MODIFY THE 20 TO BE WITH VARIABLE
    VAL_LOSS[k] = np.zeros((PARAMETERS['n_iteration'], PARAMETERS['n_epoch']))

loss_mean = np.zeros((len(PARAMETERS['decay_value']), PARAMETERS['n_epoch'] )) #rows = n. of epochs; columns = n. of decay values
val_loss_mean = np.zeros((len(PARAMETERS['decay_value']), PARAMETERS['n_epoch'] ))


for k in range(len(PARAMETERS['decay_value'])):   #scan along all the decay values
    for iteration in range(PARAMETERS['n_iteration']): #scan along all the iterations
        LOSS[k][iteration,:] = np.loadtxt("/scratch/dwaithe/"+ str(Name_file)+ "LOSS_Iter_"+ str(iteration)+ "_DecVal_" + str(PARAMETERS['decay_value'][k])+ ".txt")
        VAL_LOSS[k][iteration,:] = np.loadtxt("/scratch/dwaithe/"+ str(Name_file)+ "VAL_LOSS_Iter_"+ str(iteration)+ "_DecVal_" + str(PARAMETERS['decay_value'][k])+ ".txt")
        
        #AVERAGE OF THE loss
        loss_mean[k,:] = np.mean(LOSS[k], axis = 0) 
        #AVERAGE OF THE VAL_Loss
        val_loss_mean[k,:] = np.mean(VAL_LOSS[k], axis = 0) 
         

#PLOT
fig, ax = plt.subplots()
for i in range(len(PARAMETERS['decay_value'])):
    ax.plot(LOSS_MEAN[i,:],'-', linewidth=2, label = 'Loss') #+ 'Decay' + str(PARAMETERS['decay_value'][i])) 
    ax.plot(VAL_LOSS_MEAN[i,:],'o', ms=1, label = 'val_loss') #+ 'Decay' + str(PARAMETERS['decay_value'][i])) 

ax.set_xlabel('Epochs')
ax.set_title('Average LOSS and VAL_LOSS')
legend = plt.legend(loc='upper right', shadow=False, prop={'size':10})
fig.savefig('/scratch/dwaithe/' + str(Name_file)+ 'LOSS_and_VAL_LOSS.png')

In [ ]:
#Plot ACC and val_acc - all decay
ACC = {}
VAL_ACC = {}
for k in range(len(PARAMETERS['decay_value'])):
    ACC[k] = np.zeros((PARAMETERS['n_iteration'], PARAMETERS['n_epoch'])) #MODIFY THE 20 TO BE WITH VARIABLE
    VAL_ACC[k] = np.zeros((PARAMETERS['n_iteration'], PARAMETERS['n_epoch']))

acc_mean = np.zeros((len(PARAMETERS['decay_value']), PARAMETERS['n_epoch'] )) #rows = n. of epochs; columns = n. of decay values
val_acc_mean = np.zeros((len(PARAMETERS['decay_value']), PARAMETERS['n_epoch'] ))


for k in range(len(PARAMETERS['decay_value'])):   #scan along all the decay values
    for iteration in range(PARAMETERS['n_iteration']): #scan along all the iterations
        ACC[k][iteration,:] = np.loadtxt("/scratch/dwaithe/"+ str(Name_file)+ "ACC_Iter_"+ str(iteration)+ "_DecVal_" + str(PARAMETERS['decay_value'][k])+ ".txt")
        VAL_ACC[k][iteration,:] = np.loadtxt("/scratch/dwaithe/"+ str(Name_file)+ "VAL_ACC_Iter_"+ str(iteration)+ "_DecVal_" + str(PARAMETERS['decay_value'][k])+ ".txt")
        
        #AVERAGE OF THE ACC
        acc_mean[k,:] = np.mean(ACC[k], axis = 0) 
        #AVERAGE OF THE VAL_ACC
        val_acc_mean[k,:] = np.mean(VAL_ACC[k], axis = 0) 
         

#PLOT
fig, ax = plt.subplots()
for i in range(len(PARAMETERS['decay_value'])):
    ax.plot(acc_mean[i,:],'-', linewidth=2, label = 'Acc'+'Decay001') #)+str(decay_val[i]))
    ax.plot(val_acc_mean[i,:],'o', ms=1, label = 'val_Acc'+'Decay001') #+str(decay_val[i]))

ax.set_xlabel('Epochs')
ax.set_title('Average ACC and VAL_ACC')
legend = plt.legend(loc='upper right', shadow=False, prop={'size':10})
fig.savefig('/scratch/dwaithe/' + str(Name_file)+ 'ACC_and_VAL_ACC.png')

In [ ]:
percentage = np.zeros(1*5)
k=0
for i in range(len(PARAMETERS['decay_value'])):
    for iteration in range(PARAMETERS['n_iteration']):
    #print('Iteration = ', iteration)
        #print('Decay value =', decay_val[i])
        percentage[k] = float(np.sum(np.argmax(out[iteration][i],1)==test_cl[iteration]))/float(test_cl[iteration].shape[0])*100.0
        #print('Prediction percentage', percentage)
        k=k+1
print(percentage)
np.savetxt("/scratch/dwaithe/"+ str(Name_file)+ "_Prediction_percentage_NEW.txt", percentage,  fmt='%.4e', newline='\r\n')


In [ ]:
# Create plots with pre-defined labels.
fig, ax = plt.subplots()
for i in range(15,18):
    if test_cl[0,i] == 1:
        ax.plot(test_dt[i,:,0], label='Trig+Sing')
    if test_cl[0,i] == 2:
        ax.plot(test_dt[i,:,0], label='Trig+Osc')
    if test_cl[0,i] == 3:
        ax.plot(test_dt[i,:,0], label='Trig+Sust')
    if test_cl[0,i] == 4:
        ax.plot(test_dt[i,:,0], label='No-Trig')
    if test_cl[0,i] == 5:
        ax.plot(test_dt[i,:,0], label='Not Sure')
    legend = plt.legend(loc='upper right', shadow=True, fontsize='x-large')
